## Nested Cross Validation for GNN

Notes on the dataset and model.

- Data: subset of Lipophilicity data obtained from MoleculeNet (https://moleculenet.org/datasets-1).
- Total of 840 datapoints are used.   
- lipophilicity is a continuous number, hence the problem is framed as regression.   
- A model from torch_geometric is used for experiment (AttentiveFP based on graph attention), Mean squared error (MSE) is used as loss function and evaluation.
- Training parameters such as number of K-folds for outer and inner CV, number of trials for hyper-parameter tuning, number of training epochs are set to small values here for demo purposes.
- Final training logs are saved in separate .log file. One example is uploaded under the same repo.
- This notebook is used on Google Colab for training.

About nested CV:
- Outer CV is used for evaluation of model.
- Inner CV is used for h-param tuning using train_data within a outer CV fold. The set of h-params is used to train model and evaluate on all inner-cv folds.
- The best set of h-params (best mean score across inner CV) is used to train a model on the outer CV train set, and evaluated on the outer CV test set.


In [1]:
import math
import numpy as np
import pandas as pd
from pathlib import Path
import os
from typing import List

import torch
from torch.utils.data import SubsetRandomSampler
import torch_geometric
from torch_geometric.nn import AttentiveFP
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool
from torch_geometric.data import Dataset
from torch_geometric.utils import from_smiles

from sklearn.model_selection import KFold, StratifiedKFold
from torch.nn import MSELoss
from functools import partial

import optuna

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# ! pip install torch_geometric
# ! pip install rdkit
# ! pip install sklearn
# ! pip install optuna

### Custom molecule dataset class

- Class uses from_smiles function from torch.geometric to create graph data.

In [4]:
class MoleculeDataset(Dataset):
    """Create a custom molecule dataset with filenames (list of filenames)

    Attributes:
      filenames: list of input dataset files (csv)
      smiles_label: the column name of the smiles col
      target_label: the target column name to be predicted (y label)
      data_count: the total number of datapoints (not needed in initialization)
    """
    def __init__(self, root, filenames:List[str], smiles_label:str, target_label:str, transform=None, pre_transform=None, pre_filter=None):
        self.filenames = filenames
        self.smiles_label = smiles_label
        self.target_label = target_label
        self.data_count = None
        super(MoleculeDataset, self).__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return self.filenames

    @property
    def processed_file_names(self):
        if self.data_count is None:
            self.data_count = self._calculate_num_data_objects()

        return [f'data_{i}.pt' for i in range(self.data_count)]

    def _calculate_num_data_objects(self): #calculate once when initializing
        count = 0
        for raw_path in self.raw_paths:
            df = pd.read_csv(raw_path)
            count += len(df)
        return count

    def download(self):
        pass

    def process(self):
        idx = 0
        for raw_path in self.raw_paths:
            df = pd.read_csv(self.raw_paths[0]).reset_index()
            for i, smile in enumerate(df[self.smiles_label]):
              g = from_smiles(smile)
              g.x = g.x.float()
              y = torch.tensor(df[self.target_label][i], dtype=torch.float).view(1, -1)
              g.y = y

              torch.save(g, os.path.join(self.processed_dir, f'data_{idx}.pt'))
              idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt'))
        return data

### Train, validate and test functions

In [5]:
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

def train_one_epoch(model:torch_geometric.nn, train_loader:DataLoader, optimizer:torch.optim, loss_fn:torch.nn) -> float:
    """
    Train a torch geometric model for 1 epoch.
    Args:
      model: torch_geometric.nn model to be trained
      train_loader: DataLoader for train data
      optimizer: torch optimizer
      loss_fn: loss function from torch.nn

    Returns:
      loss_ep: loss for current epoch (float)
    """
    running_loss, num_data = 0,0
    for _, data in enumerate(train_loader):
        data = data.to(device)
        # reset grad
        optimizer.zero_grad()
        ## get pred from model
        pred = model(data.x, data.edge_index, data.edge_attr, data.batch)
        # get loss and grads
        loss = loss_fn(pred, data.y)
        loss.backward()
        optimizer.step()

        ## mse need to take average across number of samples.
        ## num of data in last batch may not be same as batch size.
        running_loss += float(loss) * data.num_graphs ##expand loss on batch by batch size
        num_data += data.num_graphs
    loss_ep = running_loss/num_data
    return loss_ep

def train(num_epochs:int, model:torch_geometric.nn, train_loader:DataLoader, optimizer:torch.optim, loss_fn:torch.nn):
    """
    Train a torch geometric model for specified epoch numbers
    Args:
      num_epochs: num epochs
      model: torch_geometric.nn model to be trained
      train_loader: DataLoader for train data
      optimizer: torch optimizer
      loss_fn: loss function from torch.nn

    Returns:
      None
    """
    model.train()
    for ep in range(num_epochs):
        loss = train_one_epoch(model, train_loader, optimizer, loss_fn)
        logger.info(f"Epoch {ep} | Train Loss (MSE) {loss}")
        ## can add mlflow metrics logging / early stopping here

@torch.no_grad()
def validate(model:torch_geometric.nn, loader:DataLoader, loss_fn:torch.nn) -> float:
    """
    Compute validation loss for model
    Args:
      model: torch_geometric.nn model to be validated
      loader: DataLoader for validation data å
      loss_fn:l oss function from torch.nn

    Returns:
      valid_loss: validation loss (float)
    """
    valid_loss = 0
    num_data = 0
    model.eval()
    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = loss_fn(pred, data.y)
        valid_loss += float(loss) * data.num_graphs ##expand loss on batch by batch size
        num_data += data.num_graphs
    valid_loss = valid_loss/num_data
    return valid_loss


@torch.no_grad()
def test(model:torch_geometric.nn, loader:DataLoader, loss_fn:torch.nn) -> float:
    """
    Test/Evaluate the model
    Args:
      model: torch_geometric.nn model to be validated
      loader: DataLoader for validation data
      loss_fn: loss function from torch.nn

    Returns:
      mse: returns mse (float)
    """
    output = []
    model.eval()
    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = loss_fn(pred, data.y)
        concatenated_batch = torch.cat((pred, data.y), dim=1) ## dim = [num_test, 2]
        output.append(concatenated_batch)

    # Stack the tensors along batch of data dimension
    stacked_output = torch.cat(output, dim=0) #concat by rows - all data in batch

    ## can add additional evaluation metrics and/or return actual output for plotting
    mse = mean_squared_error(stacked_output[1], stacked_output[0])
    logger.info(f"Test Evaluation MSE: {mse}")

    return mse


### Inner CV for h-param optimization

In [6]:
def cv_objective(trial:int, k_inner:int, model:torch_geometric.nn, train_val_data:Dataset, loss_fn:torch.nn, num_epochs:int):
    """
    Inner CV to perform h-param optimization.
    Args:
      trial: trial number
      k_inner: number of inner CV folds
      model: torch_geometric.nn model to be trained
      train_val_data: Dataset containing train and validation data for inner CV
      loss_fn: loss function from torch.nn

    Returns: mean of inner cv loss

    """
    logger.info(f'Performing inner CV hparam optimization')
    fold = KFold(n_splits=k_inner, shuffle=True, random_state=0)
    inner_cv_loss = []
    # get hparams for this trial
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=False)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=False)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                weight_decay=weight_decay)
    for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(range(len(train_val_data)))):
        train_data = torch.utils.data.Subset(dataset, train_idx)
        valid_data = torch.utils.data.Subset(dataset, valid_idx)

        train_loader = DataLoader(
            train_data,
            batch_size=32,
            shuffle=True
        )
        valid_loader = DataLoader(
            valid_data,
            batch_size=32,
            shuffle=True,
        )
        ## reset model
        model.reset_parameters()
        # train model
        train(num_epochs, model, train_loader, optimizer, loss_fn)
        # get valid loss
        loss_v = validate(model, valid_loader, loss_fn)
        inner_cv_loss.append(loss_v)
    return np.mean(inner_cv_loss)


### Main Outer CV

In [7]:
import logging
def nested_cv_experiment(config:dict):
    """
    Run the nested CV experiment using config dictionary
    Args:
      config: config for the experiment, including model, dataset, loss, optimizer, k-folds settings and epochs.
    """
    ## training experiment settings unpack
    model = config['model']
    dataset = config['dataset']
    loss_fn = config['loss_fn']
    optimizer_class = config['optimizer_class']
    num_epochs = config['num_epochs']
    batch_size = config['batch_size']
    k_inner = config['k_inner']
    k_outer = config['k_outer']
    objective = config['objective']
    n_trials = config['n_trials']
    timeout_optuna = config['timeout_optuna']

    cv_outer = KFold(n_splits=k_outer, shuffle=True, random_state=22)
    eval_loss = []
    ## Outer CV
    for fold, (train_ids, test_ids) in enumerate(cv_outer.split(range(len(dataset)))):
        logger.info(f"{fold}-FOLD out of {k_outer}")
        logger.info(f"Train length: {len(train_ids)} Validate length: {len(test_ids)} SUM:, {len(test_ids)+len(train_ids)}")
        train_data = torch.utils.data.Subset(dataset, train_ids)
        test_data = torch.utils.data.Subset(dataset, test_ids)

        ## start inner CV for finding the best hparams on this fold
        model.to(device).reset_parameters()

        logger.info('Optimizing hparams')
        study = optuna.create_study(direction="minimize")
        cv_objective = partial(objective, k_inner=k_inner, model=model, train_val_data=train_data, loss_fn=loss_fn, num_epochs=num_epochs) # perform inner CV for h-param tuning using train data from outer CV split
        study.optimize(cv_objective, n_trials=2, timeout=timeout_optuna)

        best_params = study.best_trial.params
        logger.info("Best h-params: ", best_params)
        optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'],
                                    weight_decay=best_params['weight_decay'])

        ## retrain model on best params optimizer
        logger.info('Test on unseen data using the model+best hparams from inner loop')
        model.reset_parameters()
        train_loader = DataLoader(train_data, batch_size=32)
        train(num_epochs, model, train_loader, optimizer, loss_fn)

        test_loader = DataLoader(test_data, batch_size=32)
        mse_fold = test(model, test_loader, loss_fn)

        eval_loss.append(mse_fold)

    logger.info(f'Mean MSE across Outer CV: {np.mean(eval_loss)}')

## Run experiment

In [8]:
## Set path to data and working directory for dataset creation.
root_path = '/content/drive/MyDrive/Colab Notebooks/data_gnn/data/'
working_path = '/content/drive/MyDrive/Colab Notebooks/data_gnn/working'
data_files = []
for filename in os.listdir(root_path):
    data_files.append(root_path+filename)

print(data_files)

['/content/drive/MyDrive/Colab Notebooks/data_gnn/data/Lipophilicity.csv']


Checking dataset  

In [9]:
dataset = MoleculeDataset(root=working_path, filenames=data_files, smiles_label='smiles', target_label='exp')

dataset[0], dataset[2]

## Input data feature: in_dim=9, edge_dim=3

(Data(x=[22, 9], edge_index=[2, 48], edge_attr=[48, 3], smiles='CC(C)c1ccc2Oc3nc(N)c(cc3C(=O)c2c1)C(=O)O', y=[1, 1]),
 Data(x=[28, 9], edge_index=[2, 60], edge_attr=[60, 3], smiles='CN(C)C(=O)N[C@@H]1CC[C@@H](CCN2CCN(CC2)c3ccc(Cl)c(Cl)c3)CC1', y=[1, 1]))

In [10]:
import logging

# set experiment config
config = {
    'dataset': MoleculeDataset(root=working_path, filenames=data_files, smiles_label='smiles', target_label='exp'),
    'model': AttentiveFP(in_channels=9, hidden_channels=64, out_channels=1,
                    edge_dim=3, num_layers=2, num_timesteps=2,
                    dropout=0.2),
    'loss_fn': MSELoss(),
    'optimizer_class': torch.optim.Adam,
    'scheduler_params': {'gamma': 0.9},
    'num_epochs': 2,
    'batch_size': 32,
    'k_inner': 3,
    'k_outer': 5,
    'objective': cv_objective,
    'n_trials': 2,
    'timeout_optuna': 300
}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# get logger
log_file_path = '/content/drive/MyDrive/Colab Notebooks/data_gnn/experiment_attentiveFP_nestedcv.log'
logging.basicConfig(filename=log_file_path,
                    format='%(asctime)s %(message)s',
                    level=logging.DEBUG,
                    filemode='w',  force=True)
logger = logging.getLogger('log')

# run experiment on current config
nested_cv_experiment(config)

[I 2024-07-24 05:04:49,000] A new study created in memory with name: no-name-efee1c96-a188-4f0e-9183-7d2fa708a564
[I 2024-07-24 05:05:15,377] Trial 0 finished with value: 1.6451580779893058 and parameters: {'lr': 0.008658744473535875, 'weight_decay': 0.00025948971735842744}. Best is trial 0 with value: 1.6451580779893058.
[I 2024-07-24 05:05:34,627] Trial 1 finished with value: 1.5202242732048035 and parameters: {'lr': 0.009545491245148186, 'weight_decay': 8.527351248477438e-05}. Best is trial 1 with value: 1.5202242732048035.
[I 2024-07-24 05:05:42,987] A new study created in memory with name: no-name-082f36cb-ef23-42a8-a9c5-5ead514b0f9e
[I 2024-07-24 05:06:02,015] Trial 0 finished with value: 1.8114200433095295 and parameters: {'lr': 0.006964723383840363, 'weight_decay': 0.0006996485545470178}. Best is trial 0 with value: 1.8114200433095295.
[I 2024-07-24 05:06:21,210] Trial 1 finished with value: 1.492532233397166 and parameters: {'lr': 0.002992464149670449, 'weight_decay': 1.760985

In [11]:
# check experiment log
print(Path(log_file_path).read_text())

2024-07-24 05:04:48,996 0-FOLD out of 5
2024-07-24 05:04:48,997 Train length: 672 Validate length: 168 SUM:, 840
2024-07-24 05:04:48,999 Optimizing hparams
2024-07-24 05:04:49,002 Performing inner CV hparam optimization
2024-07-24 05:04:52,587 Epoch 0 | Train Loss (MSE) 26.069338176931655
2024-07-24 05:04:56,787 Epoch 1 | Train Loss (MSE) 1.9465930802481515
2024-07-24 05:05:03,978 Epoch 0 | Train Loss (MSE) 16.121856825692312
2024-07-24 05:05:07,680 Epoch 1 | Train Loss (MSE) 3.172085762023926
2024-07-24 05:05:11,671 Epoch 0 | Train Loss (MSE) 9.440867551735469
2024-07-24 05:05:14,148 Epoch 1 | Train Loss (MSE) 2.098649493285588
2024-07-24 05:05:15,379 Performing inner CV hparam optimization
2024-07-24 05:05:18,253 Epoch 0 | Train Loss (MSE) 49.89956702504839
2024-07-24 05:05:20,762 Epoch 1 | Train Loss (MSE) 2.0127105627741133
2024-07-24 05:05:24,295 Epoch 0 | Train Loss (MSE) 5.439251380307334
2024-07-24 05:05:26,688 Epoch 1 | Train Loss (MSE) 1.9557387573378426
2024-07-24 05:05:30,7